In [157]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import os
import seaborn as sns
import csv
import pandas as pd
import re

from glob import glob
from numpy.ma.core import squeeze
from pathlib import Path
from sklearn.metrics import confusion_matrix
from collections import Counter
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import random
from pydub import AudioSegment
from pydub.playback import play
import time
from sklearn.ensemble import RandomForestRegressor

In [158]:
pip install pydub simpleaudio

Note: you may need to restart the kernel to use updated packages.


In [159]:
data = pd.read_csv("/Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/tabla_predictions_kaggle_unlabelled_combined.csv")

In [160]:
print(data)

                Filename  Stroke_Index   Bol  Duration
0    Addhatrital-069.wav             0   Dha   0.03200
1    Addhatrital-069.wav             1  Dhin   0.38350
2    Addhatrital-069.wav             2    Na   0.26617
3    Addhatrital-069.wav             3   Dha   0.12750
4    Addhatrital-069.wav             4   Dha   0.26617
..                   ...           ...   ...       ...
976     Teentaal-099.wav            12    Ta   0.28750
977     Teentaal-099.wav            13  Dhin   0.29817
978     Teentaal-099.wav            14  Dhin   0.29817
979     Teentaal-099.wav            15   Dha   0.28750
980     Teentaal-099.wav            16     T   0.24483

[981 rows x 4 columns]


In [161]:



print(len(data))

981


In [162]:
note_labels = ["Dha", "Dhin", "Ghe", "Kat", "Ki", "Na", "Re", "T", "Ta", "Ti", "Tun", "Tin"]

In [163]:
seq_data = []
row = []
for i in range(0,len(data)):
    if data.iloc[i]['Stroke_Index'] == 0:
        if len(row) > 0:
            seq_data.append(row)
        row = []
    row.append((data.iloc[i]['Bol'],data.iloc[i]['Duration']))


In [164]:
print(seq_data[0])

[('Dha', np.float64(0.032)), ('Dhin', np.float64(0.3835)), ('Na', np.float64(0.26617)), ('Dha', np.float64(0.1275)), ('Dha', np.float64(0.26617)), ('Dhin', np.float64(0.3835)), ('Na', np.float64(0.26617)), ('Dha', np.float64(0.13817)), ('Dha', np.float64(0.2555)), ('Tin', np.float64(0.3835)), ('T', np.float64(0.2555)), ('Ta', np.float64(0.13817)), ('Na', np.float64(0.26617)), ('Dhin', np.float64(0.3835)), ('Dhin', np.float64(0.2555)), ('Dha', np.float64(0.13817))]


In [165]:
'''given three notes, predict the next note'''

'given three notes, predict the next note'

In [166]:
print(len(seq_data))

79


In [167]:
model_data = []
for i in range(0,len(seq_data)):

    for j in range(0,len(seq_data[i])-3):
        '''point'''
        row = []
        row.append(note_labels.index(seq_data[i][j][0]))
        row.append(seq_data[i][j][1])
        row.append(note_labels.index(seq_data[i][j+1][0]))
        row.append(seq_data[i][j+1][1])
        row.append(note_labels.index(seq_data[i][j+2][0]))
        row.append(seq_data[i][j+2][1])
        row.append(note_labels.index(seq_data[i][j+3][0]))
        row.append(seq_data[i][j+3][1])
        model_data.append(row)

In [168]:
print((model_data))

[[0, np.float64(0.032), 1, np.float64(0.3835), 5, np.float64(0.26617), 0, np.float64(0.1275)], [1, np.float64(0.3835), 5, np.float64(0.26617), 0, np.float64(0.1275), 0, np.float64(0.26617)], [5, np.float64(0.26617), 0, np.float64(0.1275), 0, np.float64(0.26617), 1, np.float64(0.3835)], [0, np.float64(0.1275), 0, np.float64(0.26617), 1, np.float64(0.3835), 5, np.float64(0.26617)], [0, np.float64(0.26617), 1, np.float64(0.3835), 5, np.float64(0.26617), 0, np.float64(0.13817)], [1, np.float64(0.3835), 5, np.float64(0.26617), 0, np.float64(0.13817), 0, np.float64(0.2555)], [5, np.float64(0.26617), 0, np.float64(0.13817), 0, np.float64(0.2555), 11, np.float64(0.3835)], [0, np.float64(0.13817), 0, np.float64(0.2555), 11, np.float64(0.3835), 7, np.float64(0.2555)], [0, np.float64(0.2555), 11, np.float64(0.3835), 7, np.float64(0.2555), 8, np.float64(0.13817)], [11, np.float64(0.3835), 7, np.float64(0.2555), 8, np.float64(0.13817), 5, np.float64(0.26617)], [7, np.float64(0.2555), 8, np.float64(

In [169]:
final_data = pd.DataFrame(model_data)
final_data.duplicated().sum()

np.int64(189)

In [170]:
final_data.columns = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'target_1', 'target_2']
final_data.drop_duplicates(inplace=True)

In [171]:
print(final_data.shape)

(538, 8)


In [172]:
x = final_data.drop(columns=['target_1', 'target_2'])
y_1 = final_data['target_1']
y_2 = final_data['target_2']

In [173]:
x_train, x_test, y_train, y_test = train_test_split(x, y_1, test_size=0.2, random_state=42)

In [174]:
model = RandomForestClassifier(n_estimators=10)

In [175]:
model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=10)

In [176]:
y_pred = model.predict(x_test)

In [177]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6944444444444444
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.76      0.81        29
           1       0.71      0.62      0.67        24
           2       0.00      0.00      0.00         1
           5       0.69      0.61      0.65        18
           7       0.59      0.67      0.62        15
           8       0.83      1.00      0.91         5
           9       0.80      0.67      0.73         6
          10       0.50      1.00      0.67         1
          11       0.50      0.78      0.61         9

    accuracy                           0.69       108
   macro avg       0.61      0.68      0.63       108
weighted avg       0.72      0.69      0.70       108

Confusion Matrix:
 [[22  2  1  2  2  0  0  0  0]
 [ 1 15  1  0  1  0  1  1  4]
 [ 0  0  0  1  0  0  0  0  0]
 [ 1  1  0 11  2  0  0  0  3]
 [ 0  2  0  2 10  1  0  0  0]
 [ 0  0  0  0  0  5  0  0  0]
 [ 0  0  0  0  2  0  4  0  0]
 [ 0  0  0  0

In [178]:
print(x.iloc[0])

feature_1    0.00000
feature_2    0.03200
feature_3    1.00000
feature_4    0.38350
feature_5    5.00000
feature_6    0.26617
Name: 0, dtype: float64


In [179]:
x_train_duration, x_test_duration, y_train_duration, y_test_duration = train_test_split(x, y_2, test_size=0.2, random_state=42)

In [180]:
model_2 = RandomForestRegressor(n_estimators=100, random_state=42)

In [181]:
model_2.fit(x_train_duration, y_train_duration)

RandomForestRegressor(random_state=42)

In [182]:
y_pred_duration = model_2.predict(x_test_duration)

In [183]:
loss = model_2.score(x_test_duration, y_test_duration)

In [184]:
print(loss)

0.8594293124602924


In [185]:
x_test_sample = model_data[0][0:6]
for i in range(0,3):
    print(note_labels[x_test_sample[i*2]])
predicted_notes = []
predicted_duration = []
for i in range(0,30):
    pred_note = model.predict([x_test_sample])
    pred_duration = model_2.predict([x_test_sample])
    print(note_labels[pred_note[0]], pred_duration[0])
    predicted_notes.append(note_labels[pred_note[0]])
    predicted_duration.append(pred_duration[0])
    x_test_sample = x_test_sample[2:]
    x_test_sample.append(pred_note[0])
    x_test_sample.append(pred_duration[0])

Dha
Dhin
Na
Dha 0.14341306666666667
Dha 0.2593056333333335
Tin 0.33588194166666696
T 0.26126170000000015
Ta 0.1513695250000002
Dhin 0.2580282116666668
Dhin 0.20088680000000003
Dha 0.20966583333333336
Dha 0.2847626666666668
Tin 0.2327607383333335
Dhin 0.25761791666666656
Tin 0.27952679666666663
Dha 0.25875275000000025
Tin 0.17871568000000018
Ti 0.35599311666666655
Ti 0.22508606523809527
Dha 0.29972394166666677
Dha 0.26849326000000007
Dhin 0.22197660166666672
Dhin 0.35195891
Dha 0.2239536750000002
Dha 0.21158853333333344
Dha 0.37046891666666665
Tin 0.2372517083333336
Tin 0.21532436666666668
Ta 0.3023627342857142
Ta 0.26285179166666667
Dha 0.29304720000000034
T 0.27907733499999987
Tin 0.23289603333333347


/opt/anaconda3/envs/ML312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/ML312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/ML312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/ML312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/ML312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomF

In [186]:
print(predicted_notes)
print(predicted_duration)

['Dha', 'Dha', 'Tin', 'T', 'Ta', 'Dhin', 'Dhin', 'Dha', 'Dha', 'Tin', 'Dhin', 'Tin', 'Dha', 'Tin', 'Ti', 'Ti', 'Dha', 'Dha', 'Dhin', 'Dhin', 'Dha', 'Dha', 'Dha', 'Tin', 'Tin', 'Ta', 'Ta', 'Dha', 'T', 'Tin']
[np.float64(0.14341306666666667), np.float64(0.2593056333333335), np.float64(0.33588194166666696), np.float64(0.26126170000000015), np.float64(0.1513695250000002), np.float64(0.2580282116666668), np.float64(0.20088680000000003), np.float64(0.20966583333333336), np.float64(0.2847626666666668), np.float64(0.2327607383333335), np.float64(0.25761791666666656), np.float64(0.27952679666666663), np.float64(0.25875275000000025), np.float64(0.17871568000000018), np.float64(0.35599311666666655), np.float64(0.22508606523809527), np.float64(0.29972394166666677), np.float64(0.26849326000000007), np.float64(0.22197660166666672), np.float64(0.35195891), np.float64(0.2239536750000002), np.float64(0.21158853333333344), np.float64(0.37046891666666665), np.float64(0.2372517083333336), np.float64(0.215

In [187]:
import os
import random
import numpy as np
from pydub import AudioSegment
import simpleaudio as sa

# Set this to your dataset folder path
DATASET_PATH = "/Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN"

def get_random_sample(note):
    """Returns a random AudioSegment for a given tabla note label"""
    folder = os.path.join(DATASET_PATH, note)
    print(f"🧾 Looking in folder: {folder}")
    files = [f for f in os.listdir(folder) if f.endswith(".wav")]
    if not files:
        raise FileNotFoundError(f"No WAV files found in {folder}")
    chosen = random.choice(files)
    return AudioSegment.from_wav(os.path.join(folder, chosen))

def adjust_duration(audio, target_sec):
    """Trims or pads the audio to match the predicted duration (in seconds)"""
    target_ms = int(target_sec * 1000)
    if len(audio) > target_ms:
        return audio[:target_ms]
    else:
        silence = AudioSegment.silent(duration=target_ms - len(audio))
        return audio + silence

def play_audiosegment(audio):
    """Plays a pydub AudioSegment with blocking using simpleaudio"""
    playback = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate,
    )
    playback.wait_done()

def play_sequence(predicted_notes, predicted_durations):
    """Main sampler loop"""
    for note, dur in zip(predicted_notes, predicted_durations):
        note = str(note)
        duration = float(dur)
        try:
            sample = get_random_sample(note)
            sample = adjust_duration(sample, duration)
            print(f"Playing {note} for {round(duration, 3)} sec")
            play_audiosegment(sample)
        except Exception as e:
            print(f"Error with {note}: {e}")


In [188]:
def export_sequence(predicted_notes, predicted_durations, output_path="output.wav"):
    sequence = AudioSegment.silent(duration=0)
    for note, dur in zip(predicted_notes, predicted_durations):
        note = str(note).strip()  # Ensure no whitespace
        duration = float(dur)
        try:
            sample = get_random_sample(note)
            sample = adjust_duration(sample, duration)
            sequence += sample
        except Exception as e:
            print(f"Error with {note}: {e}")
    if len(sequence) == 0:
        print("⚠️ Nothing to export. Sequence is empty.")
        return
    sequence.export(output_path, format="wav")
    print(f"✅ Exported to {output_path}")


In [189]:
export_sequence(predicted_notes, predicted_duration)


🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Dha
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Dha
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Tin
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/T
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Ta
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Dhin
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Dhin
🧾 Looking in folder: /Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/DataSet/TablaSN/Dha
🧾 Looking in folder: /Users/shreyagupta/D

In [190]:
''' play this as drum sampler '''
'''
1 - Dha - SD + Ride
2 - Dhin - BD + Ride
3 - Ta - Ride
4 - Tin - Hihat (foot)
5 - Na - SnareLeft
6 - Ge - BD
7 - kat/ki/ke - SD
8 - T - SD
9 - Te/re -SD
10 - Ti - SD
11 - Tun - TOMB + RIDE
'''

'\n1 - Dha - SD + Ride\n2 - Dhin - BD + Ride\n3 - Ta - Ride\n4 - Tin - Hihat (foot)\n5 - Na - SnareLeft\n6 - Ge - BD\n7 - kat/ki/ke - SD\n8 - T - SD\n9 - Te/re -SD\n10 - Ti - SD\n11 - Tun - TOMB + RIDE\n'

In [191]:
import os
import random
import numpy as np
from pydub import AudioSegment
import simpleaudio as sa

# Set this to your dataset folder path
DATASET_PATH = "/Users/shreyagupta/Desktop/Machine_Learning/Group_Project/Project_Tabla_Kaggle/Drum_shots"


def get_random_sample(note):
    """Returns a random AudioSegment for a given tabla note label"""
    folder = os.path.join(DATASET_PATH, note)
    return AudioSegment.from_wav(os.path.join(folder))


def adjust_duration(audio, target_sec):
    """Trims or pads the audio to match the predicted duration (in seconds)"""
    target_ms = int(target_sec * 1000)
    if len(audio) > target_ms:
        return audio[:target_ms]
    else:
        silence = AudioSegment.silent(duration=target_ms - len(audio))
        return audio + silence


def play_audiosegment(audio):
    """Plays a pydub AudioSegment with blocking using simpleaudio"""
    playback = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate,
    )
    playback.wait_done()


def play_sequence(predicted_notes, predicted_durations):
    """Main sampler loop"""
    for note, dur in zip(predicted_notes, predicted_durations):
        note = str(note)
        duration = float(dur)
        try:
            sample = get_random_sample(note)
            sample = adjust_duration(sample, duration)
            print(f"Playing {note} for {round(duration, 3)} sec")
            play_audiosegment(sample)
        except Exception as e:
            print(f"Error with {note}: {e}")


def export_sequence(predicted_notes, predicted_durations, output_path="drum_output.wav"):
    sequence = AudioSegment.silent(duration=0)
    for note, dur in zip(predicted_notes, predicted_durations):
        note = str(note).strip()  # Ensure no whitespace
        note_wav = note + ".wav"
        duration = float(dur)
        try:
            sample = get_random_sample(note_wav)
            sample = adjust_duration(sample, duration)
            sequence += sample
        except Exception as e:
            print(f"Error with {note_wav}: {e}")
    if len(sequence) == 0:
        print("⚠️ Nothing to export. Sequence is empty.")
        return
    sequence.export(output_path, format="wav")
    print(f"✅ Exported to {output_path}")

In [192]:
# After playing the tabla prediction
print(predicted_notes)
print(predicted_duration)
export_sequence(predicted_notes, predicted_duration)

['Dha', 'Dha', 'Tin', 'T', 'Ta', 'Dhin', 'Dhin', 'Dha', 'Dha', 'Tin', 'Dhin', 'Tin', 'Dha', 'Tin', 'Ti', 'Ti', 'Dha', 'Dha', 'Dhin', 'Dhin', 'Dha', 'Dha', 'Dha', 'Tin', 'Tin', 'Ta', 'Ta', 'Dha', 'T', 'Tin']
[np.float64(0.14341306666666667), np.float64(0.2593056333333335), np.float64(0.33588194166666696), np.float64(0.26126170000000015), np.float64(0.1513695250000002), np.float64(0.2580282116666668), np.float64(0.20088680000000003), np.float64(0.20966583333333336), np.float64(0.2847626666666668), np.float64(0.2327607383333335), np.float64(0.25761791666666656), np.float64(0.27952679666666663), np.float64(0.25875275000000025), np.float64(0.17871568000000018), np.float64(0.35599311666666655), np.float64(0.22508606523809527), np.float64(0.29972394166666677), np.float64(0.26849326000000007), np.float64(0.22197660166666672), np.float64(0.35195891), np.float64(0.2239536750000002), np.float64(0.21158853333333344), np.float64(0.37046891666666665), np.float64(0.2372517083333336), np.float64(0.215

In [193]:
import pickle

# save
with open('model.pkl','wb') as f:
    pickle.dump(model,f)

with open('model_2.pkl','wb') as f:
    pickle.dump(model_2,f)


✅ Exported to drum_output.wav
